In [40]:
import pandas as pd
import psycopg2

In [41]:
server = 'localhost'
database = 'python_etl'
user = 'postgres'
password = 'root'

connection_string = f"dbname={database} user={user} password={password} host={server}"

try:
    conn = psycopg2.connect(connection_string)
    print("Conexão bem-sucedida ao PostgreSQL!")
    
    cursor = conn.cursor()
except Exception as e:
    print("Erro ao conectar ao PostgreSQL:", e)

Conexão bem-sucedida ao PostgreSQL!


In [42]:
dados = pd.read_csv("/home/juniorsilvadev/engenharia-dados/03-python-etl-do-basico-ao-avancado/Origem/Origem/arquivos_csv/Clientes.csv", delimiter=',')

In [43]:
# Converter a coluna 'created_at' para data e hora
dados['created_at'] = pd.to_datetime(dados['created_at'])

dados['email'] = dados['email'].fillna('Sem registro')
dados['street'] = dados['street'].fillna('Sem infor')
dados['number'] = dados['number'].fillna('Sem número')
dados['additionals'] = dados['additionals'].fillna('Sem infor')

In [44]:
str(dados.columns).replace("'", "")

'Index([id, created_at, first_name, last_name, email, cell_phone,\n       country, state, street, number, additionals],\n      dtype=object)'

In [45]:
create_table_query = """
CREATE TABLE IF NOT EXISTS clientes (
    id SERIAL PRIMARY KEY,
    created_at TIMESTAMP,
    first_name VARCHAR(255),
    last_name VARCHAR(255),
    email VARCHAR(255),
    cell_phone VARCHAR(255),
    country VARCHAR(255),
    state VARCHAR(255),
    street VARCHAR(255),
    number VARCHAR(255),
    additionals VARCHAR(255)
);
"""

try:
    cursor.execute(create_table_query)
    conn.commit()
    print("Tabela 'ordens' criada com sucesso!")
except Exception as e:
    print("Erro ao criar a tabela 'categorias':", e)
    conn.rollback()

Tabela 'ordens' criada com sucesso!


In [46]:
cursor.execute("TRUNCATE TABLE clientes")

In [47]:
insert_query = """
INSERT INTO clientes (id, created_at, first_name, last_name, email, cell_phone, country, state, street, number, additionals) 
VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
"""

try:
    for index, row in dados.iterrows():
        cursor.execute(insert_query, (
            row['id'], 
            row['created_at'],
            row['first_name'],
            row['last_name'],
            str(row['email']),
            row['cell_phone'],
            str(row['country']),
            str(row['state']),
            str(row['street']),
            str(row['number']),
            str(row['additionals']),
        ))
    
    conn.commit()
    print("Dados inseridos com sucesso!")
except Exception as e:
    print("Erro ao inserir dados:", e)
    conn.rollback()
finally:
    cursor.close()
    conn.close()

Dados inseridos com sucesso!
